# Analyse de Series Temporelles Climatiques

Ce notebook presente une analyse complete de series temporelles climatiques (temperature et CO2) avec modelisation predictive et evaluation de robustesse.

## Objectifs
1. Charger et preparer les donnees climatiques
2. Effectuer une analyse exploratoire
3. Construire des modeles predictifs
4. Evaluer la robustesse des modeles
5. Generer des visualisations professionnelles

In [ ]:
# Imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Modules personnalises
from data_loader import prepare_climate_data
from analysis import (plot_time_series, decompose_series, plot_decomposition, 
                      compute_correlation, plot_correlation_matrix, plot_scatter_with_trend,
                      analyze_trends)
from modeling import (split_temporal_data, prepare_features, train_linear_model, 
                      train_polynomial_model, evaluate_model, plot_predictions,
                      plot_residuals, compare_models)
from robustness import (test_noise_sensitivity, plot_robustness_results,
                        evaluate_on_windows, plot_window_performance, compute_stability_metrics)

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

# Creer les dossiers de resultats
Path('../results/figures').mkdir(parents=True, exist_ok=True)
Path('../results/metrics').mkdir(parents=True, exist_ok=True)

print('Environnement configure')

## 1. Chargement et Preparation des Donnees

In [ ]:
# Charger les donnees
data_path = '../data/raw/climate_data.csv'
df = prepare_climate_data(data_path, normalize=True)

print(f'Donnees chargees: {len(df)} observations')
print(f'Periode: {df.index.min()} a {df.index.max()}')
print('\nApercu des donnees:')
display(df.head())
print('\nStatistiques descriptives:')
display(df.describe())

## 2. Analyse Exploratoire

In [ ]:
# Visualiser les series temporelles brutes
plot_time_series(df, ['temperature', 'co2'], 
                'Evolution Temporelle - Temperature et CO2',
                save_path='../results/figures/01_series_temporelles.png')

In [ ]:
# Decomposition de la serie temperature
decomp_temp = decompose_series(df['temperature'], period=12)
plot_decomposition(decomp_temp, 'Decomposition - Temperature',
                  save_path='../results/figures/02_decomposition_temperature.png')

In [ ]:
# Analyse de correlation
corr = compute_correlation(df, 'temperature', 'co2')
print(f'Correlation temperature-CO2: {corr:.3f}')

plot_scatter_with_trend(df, 'co2', 'temperature',
                       save_path='../results/figures/04_correlation_temp_co2.png')

## 3. Modelisation Predictive

In [ ]:
# Preparer les donnees pour la modelisation
df_model = df[['co2', 'temperature']].copy()
train_df, test_df = split_temporal_data(df_model, test_size=0.2)

X_train, y_train = prepare_features(train_df, 'temperature', ['co2'])
X_test, y_test = prepare_features(test_df, 'temperature', ['co2'])

print(f'Donnees preparees:')
print(f'  Train: {len(train_df)} observations')
print(f'  Test: {len(test_df)} observations')

In [ ]:
# Modele 1: Regression lineaire
model_linear = train_linear_model(X_train, y_train)
results_linear = evaluate_model(model_linear, X_test, y_test)

print('Modele Lineaire:')
print(f'  RMSE: {results_linear["rmse"]:.4f}')
print(f'  MAE: {results_linear["mae"]:.4f}')
print(f'  R2: {results_linear["r2"]:.4f}')

In [ ]:
# Modele 2: Regression polynomiale (degre 2)
model_poly2 = train_polynomial_model(X_train, y_train, degree=2)
results_poly2 = evaluate_model(model_poly2, X_test, y_test)

print('Modele Polynomial (degre 2):')
print(f'  RMSE: {results_poly2["rmse"]:.4f}')
print(f'  MAE: {results_poly2["mae"]:.4f}')
print(f'  R2: {results_poly2["r2"]:.4f}')

In [ ]:
# Visualiser les predictions
plot_predictions(results_poly2['actuals'], results_poly2['predictions'],
                dates=test_df.index,
                title='Predictions - Modele Polynomial (degre 2)',
                save_path='../results/figures/06_predictions_poly2.png')

## 4. Analyse de Robustesse

In [ ]:
# Test de sensibilite au bruit
noise_levels = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2]
noise_results = test_noise_sensitivity(model_poly2, X_test, y_test, 
                                      noise_levels=noise_levels, n_trials=20)

print('Resultats de sensibilite au bruit:')
for level, metrics in noise_results.items():
    print(f'\nNiveau de bruit sigma={level}:')
    print(f'  RMSE: {metrics["rmse_mean"]:.4f} +/- {metrics["rmse_std"]:.4f}')

In [ ]:
# Visualiser la robustesse au bruit
plot_robustness_results(noise_results, 
                       title='Analyse de Robustesse - Sensibilite au Bruit',
                       save_path='../results/figures/09_robustesse_bruit.png')

## 5. Resume et Conclusions

In [ ]:
print('=== ANALYSE TERMINEE ===')
print('Toutes les visualisations et metriques ont ete sauvegardees.')
print('\nConsultez:')
print('  - results/figures/ pour les visualisations')
print('  - results/summary.txt pour le resume')
print('  - results/metrics/ pour les metriques detaillees')